In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.15)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.77 sec - Loss 0.099644 - ACC 77.10% - ACC Mean 77.10% - AUC 83.66% - AUC Mean 83.66% - Deter 000
Ite 00050 - 0.13 sec - Loss 0.093518 - ACC 77.66% - ACC Mean 77.56% - AUC 82.77% - AUC Mean 83.50% - Deter 021
Ite 00100 - 0.19 sec - Loss 0.077955 - ACC 76.54% - ACC Mean 76.60% - AUC 82.29% - AUC Mean 83.13% - Deter 071
Ite 00150 - 0.09 sec - Loss 0.054687 - ACC 76.00% - ACC Mean 75.94% - AUC 83.71% - AUC Mean 83.50% - Deter 028
Ite 00200 - 0.13 sec - Loss 0.043290 - ACC 77.45% - ACC Mean 76.90% - AUC 83.47% - AUC Mean 83.58% - Deter 078
Ite 00250 - 0.10 sec - Loss 0.040481 - ACC 78.18% - ACC Mean 77.87% - AUC 83.75% - AUC Mean 83.63% - Deter 128
Ite 00300 - 0.10 sec - Loss 0.039599 - ACC 77.49% - ACC Mean 77.79% - AUC 83.19% - AUC Mean 83.59% - Deter 009
Ite 00350 - 0.10 sec - Loss 0.039115 - ACC 78.25% - ACC Mean 78.05% - AUC 84.09% - AUC Mean 83.76% - Deter 025
Ite 00400 - 0.11 sec - Loss 0.038733 - ACC 78.51% - ACC Mean 78.23% - AUC 83.33% - AUC Mean 84.22% - Deter 003
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 79.76% +- 0.0% - auc: 83.68% +- 0.0%
MEAN - acc: 79.2% +- 0.0% - auc: 85.05% +- 0.0%
KNN - acc: 78.97% +- 0.0% - auc: 86.57% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:38<00:00, 130.93it/s]



GAIN RUN 1/10 - acc: 78.48% +- 3.18% - auc: 84.17% +- 5.45%



100%|██████████| 5000/5000 [00:39<00:00, 126.68it/s]



GAIN RUN 2/10 - acc: 74.1% +- 7.28% - auc: 79.97% +- 4.27%



100%|██████████| 5000/5000 [00:32<00:00, 152.10it/s]


...


GAIN RUN 8/10 - acc: 75.05% +- 4.96% - auc: 80.76% +- 3.51%



100%|██████████| 5000/5000 [00:35<00:00, 141.83it/s]



GAIN RUN 9/10 - acc: 75.67% +- 4.42% - auc: 81.39% +- 5.51%



100%|██████████| 5000/5000 [00:36<00:00, 138.34it/s]



GAIN RUN 10/10 - acc: 78.85% +- 2.49% - auc: 84.64% +- 1.87%

GAIN GLOBAL - acc: 77.65% +- 1.97% - auc: 83.69% +- 2.06%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 78.03% +- 0.0% - auc: 86.72% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 1/10 - acc: 77.26% +- 3.94% - auc: 83.98% +- 5.03%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 2/10 - acc: 78.15% +- 3.04% - auc: 85.04% +- 3.42%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9

MISSFOREST RUN 3/10 - acc: 78.89% +- 2.48% - auc: 86.22% +- 3.69%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 4/10 - acc: 78.36% +- 3.04% - auc: 83.98% +- 4.39%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9

MISSFOREST RUN 5/10 - acc: 75.72% +- 3.19% - auc: 83.21% +- 5.01%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFORES

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 76.68% +- 5.47% - auc: 83.38% +- 3.43%
SOFTIMPUTE RUN 2/10 - acc: 75.31% +- 4.38% - auc: 82.71% +- 4.09%
SOFTIMPUTE RUN 3/10 - acc: 72.78% +- 2.52% - auc: 82.19% +- 2.41%
SOFTIMPUTE RUN 4/10 - acc: 74.31% +- 3.4% - auc: 81.71% +- 2.31%
SOFTIMPUTE RUN 5/10 - acc: 74.97% +- 4.47% - auc: 81.35% +- 4.53%
SOFTIMPUTE RUN 6/10 - acc: 79.43% +- 4.97% - auc: 84.74% +- 3.02%
SOFTIMPUTE RUN 7/10 - acc: 80.91% +- 1.9% - auc: 88.36% +- 2.96%
SOFTIMPUTE RUN 8/10 - acc: 77.66% +- 5.04% - auc: 84.65% +- 2.79%
SOFTIMPUTE RUN 9/10 - acc: 78.34% +- 4.74% - auc: 83.36% +- 4.47%
SOFTIMPUTE RUN 10/10 - acc: 74.8% +- 5.27% - auc: 81.69% +- 5.24%

SOFTIMPUTE GLOBAL - acc: 76.52% +- 2.41% - auc: 83.41% +- 2.0%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.4523

SINKHORN RUN 1/10 - acc: 78.6% +- 2.93% - auc: 82.03% +- 3.74%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3807

SINKHORN RUN 2/10 - acc: 77.89% +- 5.09% - auc: 82.22% +- 4.71%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3245

SINKHORN RUN 3/10 - acc: 79.8% +- 2.75% - auc: 84.44% +- 2.75%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3335

SINKHORN RUN 4/10 - acc: 77.85% +- 2.73% - auc: 84.4% +- 4.97%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.4605

SINKHORN RUN 5/10 - acc: 81.32% +- 3.45% - auc: 86.12% +- 5.14%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3736

SINKHORN RUN 6/10 - acc: 77.3% +- 3.11% - auc: 83.74% +- 3.66%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.2752

SINKHORN RUN 7/10 - acc: 78.55% +- 5.9% - auc: 84.03% +- 6.12%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3199

SINKHORN RUN 8/10 - acc: 79.49% +- 2.9% - auc: 84.78% 

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 76.85% +- 5.43% - auc: 83.72% +- 4.48%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 76.14% +- 4.78% - auc: 83.54% +- 1.61%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 78.82% +- 4.55% - auc: 86.38% +- 2.74%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 76.15% +- 3.76% - auc: 84.64% +- 4.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 76.91% +- 4.68% - auc: 84.26% +- 2.79%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 79.48% +- 4.28% - auc: 84.7% +- 2.79%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 76.11% +- 4.52% - auc: 83.82% +- 3.17%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 80.91% +- 4.42% - auc: 85.46% +- 3.49%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - ac

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is even with MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is even with KNN
Metric AUC - OUR METHOD is even with KNN
Metric ACC - OUR METHOD is even with GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is even with MICE
Metric AUC - OUR METHOD is even with MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is even with SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
